In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var body = document.body,  
    attribs = body.attributes;
var command = "theNotebook = " + "'"+attribs['data-notebook-name'].value+"'";
kernel.execute(command);

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
try:
    run_once
except NameError:
    run_once = False
if not run_once:
    run_once = True
    
    import time
    import logging
    reload(logging)
    FORMAT = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    logpath = '../logs/%s.log' % ('.'.join(theNotebook.split('.')[:-1],))
    logging.basicConfig(filename=logpath,level=logging.DEBUG, format=FORMAT)
    print("logging to %s" % (logpath))
    logger = logging.getLogger()
    #logger.basicConfig(filename='/notebooks/Export Microbiome to database.log',level=logging.DEBUG)
    logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)

    # create formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    # add formatter to ch
    ch.setFormatter(formatter)

    # add ch to logger
    logger.addHandler(ch)

In [ ]:
%matplotlib inline
import pandas, pandas.io
import re
import seaborn as sns
import math
import scipy, scipy.stats
import matplotlib.pyplot as plt
import numpy as np
import string
import networkx as nx
import community
logging.getLogger('boto').setLevel(logging.INFO)
logging.getLogger('p100').setLevel(logging.INFO)

In [ ]:
def select(res, substring=None, ids = None):
    """Helper function to search network dataframe.
    """
    if substring:
        ss_sel = _select_substring(res,  substring.lower() )
    if ids:
        raise Exception("id unimplemented")
        id_sel = _select_ids(res, ids[0], ids[1])
    return res[ss_sel]

def _select_ids( res, datasource_id, variable_id ):
    sel = (res.ds_id_1 == datasource_id & res.variable_id_1 == variable_id )
    sel = sel | (res.ds_id_2 == datasource_id & res.variable_id_2 == variable_id )
    return sel
def _select_substring( res, substring ):
    
    ann = res.annotations_1.unique().tolist() + res.annotations_2.unique().tolist()
    sel = res.annotations_1 == ''
    text =  set([a for a in ann if a.lower().find(substring) >= 0.0])
    for an in text:
        sel = sel | (res.annotations_1 == an)
        sel = sel | (res.annotations_2 == an) 
    return sel

In [ ]:
res = pandas.read_pickle('data/community/full.DELTA.correlation.network.pkl')

In [ ]:
max_pval_adj = 0.05

sig = res[(res.pval_adj < max_pval_adj)]
sig.shape

In [ ]:
sub = select(sig, 'COACH')
remove = sub.index
mask1 = sig.index.isin(remove)
print mask1.sum()
sub = select(sig, 'AGES')
remove = sub.index
mask2 = sig.index.isin(remove)
print mask2.sum()
sub = select(sig, 'CHEMS.None.Genova.height')
remove = sub.index
mask3 = sig.index.isin(remove)
print mask3.sum()
sig = sig[~(mask1 | mask2 | mask3)]
sig.shape

In [ ]:
rem = pandas.read_csv('data/community/duplicate_measurements.05012016.txt', sep='\t', header=None)
rem.columns = ['ann1', 'ann2']

mask = []
for index, row in sig.iterrows():
    
    found = False
    for index2, row2 in rem.iterrows():
        
        if (row['annotations_1']==row2['ann1']) and (row['annotations_2']==row2['ann2']):
            found = True
            break;
        elif (row['annotations_1']==row2['ann2']) and (row['annotations_2']==row2['ann1']):
            found = True
            break;

    mask.append(found)
    
mask = np.array(mask)
sig = sig[~mask]
sig.shape

In [ ]:
# Save resulting network to text file
sig.to_pickle('data/community/correlation_network.DELTA.sig.nodups.FULL.pkl')

In [ ]:
mask = []
# Now remove all correlations between pairwise elements
for index, row in sig.iterrows():
    
    cat1 = row['annotations_1'][0:5]
    cat2 = row['annotations_2'][0:5]
    
    if (cat1 == cat2):
        mask.append(False)
    else:
        mask.append(True)
        
sig = sig[mask]
print sig.shape

In [ ]:
print select(sig, 'GENOM').shape
print select(sig, 'MICRO').shape
print select(sig, 'PROTE').shape
print select(sig, 'METAB').shape
print select(sig, 'CHEMS').shape

In [ ]:
# Save resulting network to text file
sig.to_pickle('data/community/correlation_network.DELTA.sig.nodups.INTRAOMIC.pkl')

In [ ]:
# Load data into networkx
g = nx.Graph()

In [ ]:
# add a set of nodes
nodes = sig['annotations_1'].append(sig['annotations_2']).unique()
datatype = {}
super_pathways = {}
sub_pathways = {}

# Loop over nodes and get type
for e in nodes:
    tokens = e.split('.')

    datatype[e] = tokens[0]
    if (datatype[e]=='METAB'):
        super_pathways[e] = tokens[2]
        sub_pathways[e] = tokens[3]
    
g.add_nodes_from(list(nodes))
nx.set_node_attributes(g, 'type', datatype)
nx.set_node_attributes(g, 'super_pathway', super_pathways)
nx.set_node_attributes(g, 'sub_pathway', sub_pathways)

In [ ]:
# Add a set of edges
edges = []
attributes = []
for row in sig.iterrows():
    g.add_edge(row[1]['annotations_1'], row[1]['annotations_2'], {'weight':abs(row[1]['coefficient']), 'non_abs_weight':row[1]['coefficient']})  

In [ ]:
# Count number of nodes and edges
print g.number_of_nodes(), g.number_of_edges()

In [ ]:
import sys
def girvan_newman(G, k=None, weight=None):
    """Find communities in graph using Girvan–Newman method.

    Parameters
    ----------
    G : NetworkX graph

    weight : string, optional (default=None)
       Edge data key corresponding to the edge weight.

    Returns
    -------
    List of tuples which contains the clusters of nodes.

    Examples
    --------
    >>> G = nx.path_graph(10)
    >>> comp = girvan_newman(G)
    >>> comp[0]
    ([0, 1, 2, 3, 4], [8, 9, 5, 6, 7])

    Notes
    -----
    The Girvan–Newman algorithm detects communities by progressively removing
    edges from the original graph. Algorithm removes edge with the highest
    betweenness centrality at each step. As the graph breaks down into pieces,
    the tightly knit community structure is exposed and result can be depicted
    as a dendrogram.
    """
    # The copy of G here must include the edge weight data.
    g = G.copy().to_undirected()
    components = []
    while g.number_of_edges() > 0:
        print "Number of edges", g.number_of_edges()
        sys.stdout.flush()
        _remove_max_edge(g, k, weight)
        components.append(tuple(list(H)
                                for H in nx.connected_component_subgraphs(g)))
    return components

def _remove_max_edge(G, k=None, weight=None):
    """
    Removes edge with the highest value on betweenness centrality.

    Repeat this step until more connected components than the connected
    components of the original graph are detected.

    It is part of Girvan–Newman algorithm.

    :param G: NetworkX graph
    :param weight: string, optional (default=None) Edge data key corresponding
    to the edge weight.
    """
    number_components = nx.number_connected_components(G)
    #print number_components
    while nx.number_connected_components(G) <= number_components:
        betweenness = nx.edge_betweenness_centrality(G, k=k, weight=weight)
        #print "finished betweenness", nx.number_connected_components(G), number_components
        max_value = max(betweenness.values())
        #print "max_value", max_value
        # Use a list of edges because G is changed in the loop
        for edge in list(G.edges()):
            if betweenness[edge] == max_value:
                G.remove_edge(*edge)

In [ ]:
# Note: this takes about an hou
communities = girvan_newman(g, weight='weight')

# Save the community to a file
cindex = 0
gindex = 0

with open('data/community/generated.communities.DELTA.sig.nodups.INTRAOMIC.txt', 'w') as f:

    cindex = 0
    for community in communities:
        gindex = 0
        for graph in community:

            for node in graph:
                f.write("%d\t%d\t%s\n"%(cindex, gindex, node))

            gindex += 1

        cindex += 1



In [ ]:
len(communities)

In [ ]:
# Load the community
communities_dict = []

current_community = None
current_subgraph = None
data = {}

# Load the community as a set of dictionaries
with open('data/community/generated.communities.DELTA.sig.nodups.INTRAOMIC.txt', 'r') as f:

    for line in f:
        
        tokens = line.strip().split('\t')
        c = int(tokens[0])
        subgraph = int(tokens[1])
        node = tokens[2]

        if (current_community is None):
            current_community = c
            current_subgraph = subgraph
            data[node] = subgraph
            
        elif (c != current_community):          
            communities_dict.append(data)
            data = {}
            current_community = c
            current_subgraph = subgraph
            
        data[node] = subgraph
        
    #if (len(data)>0):
    #    subgraphs.append(data)
        
    #if (len(subgraphs)>0):
    #    communities_dict.append(subgraphs)
            

In [ ]:
import community
results = []
count = 0
mods = []
for c in communities_dict:
    mod = community.modularity(c, g)
    results.append((count, mod))
    mods.append(mod)
    count += 1
    
df = pandas.DataFrame(results, columns=['index', 'modularity']).sort_values('modularity', ascending=False)


In [ ]:
df.head()

In [ ]:
plt.plot(mods)

In [ ]:
# Load the community as a list of lists
communities = []

current_community = None
current_subgraph = None
data = []
subgraphs = []

# Load the community as a set of dictionaries
with open('data/community/generated.communities.DELTA.sig.nodups.INTRAOMIC.txt', 'r') as f:

    for line in f:
        
        tokens = line.strip().split('\t')
        c = int(tokens[0])
        subgraph = int(tokens[1])
        node = tokens[2]

        if (current_community is None):
            current_community = c
            current_subgraph = subgraph
            
        else:
            
            if (subgraph != current_subgraph):
                subgraphs.append(data)
                data = []
                current_subgraph = subgraph
            
            if (c != current_community):          
                communities.append(subgraphs)
                subgraphs = []
                current_community = c
            
        data.append(node)
        
    if (len(data)>0):
        subgraphs.append(data)
        
    if (len(subgraphs)>0):
        communities.append(subgraphs)
            

In [ ]:
COMMUNITY_LEVEL = 20
nnodes = []
nedges = []
for e in communities[COMMUNITY_LEVEL]:
    if (len(e)>1):   
        sg = g.subgraph(e)
        nnodes.append(sg.number_of_nodes())
        nedges.append(sg.number_of_edges())
        
print "Number of communities", len(nnodes), len(nedges)
print sorted(nnodes)
print "Node stats", np.mean(nnodes), np.max(nnodes), np.min(nnodes), np.sum(nnodes)
print "Edge stats", np.mean(nedges), np.max(nedges), np.min(nedges), np.sum(nedges)

# Get percentage of nodes used
print "Percent of nodes in a community", float(np.sum(nnodes)) / g.number_of_nodes()
print "Percent of edges in a community", float(np.sum(nedges)) / g.number_of_edges()

In [ ]:
communities = pandas.read_csv('data/community/generated.communities.DELTA.sig.nodups.INTRAOMIC.txt', header=None, sep='\t')
communities.columns = ['cindex', 'gindex', 'node']
communities.head()